#   Tool Calling (Function Calling)

---

## 환경 설정 및 준비

`(1) Env 환경변수`

In [ ]:
from dotenv import load_dotenv
load_dotenv()

`(2) 기본 라이브러리`

In [ ]:
import os
from glob import glob

import re
import json

from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")


---

## **Tool Calling**

- **Tool Calling**은 LLM이 외부 시스템과 상호작용하기 위한 **함수 호출 메커니즘**

- LLM은 정의된 도구나 함수를 통해 **외부 시스템과 통신**하고 작업을 수행

- **Tool calling**은 모델이 시스템과 직접 상호작용할 수 있게 하는 기능

- **구조화된 출력**을 통해 API나 데이터베이스와 같은 시스템 요구사항 충족

- **스키마 기반 응답**으로 시스템 간 효율적 통신 가능


![Tool Calling Concept](https://python.langchain.com/assets/images/tool_calling_concept-552a73031228ff9144c7d59f26dedbbf.png)


[참조] https://python.langchain.com/docs/concepts/tool_calling/

**Tool Calling의 필요성**

| 상황 | 해결 방법 |
|------|-----------|
| **최신 정보 필요** | 웹 검색 도구로 실시간 정보 수집 |
| **계산 필요** | 계산기 도구로 정확한 연산 수행 |
| **데이터베이스 접근** | DB 도구로 정보 조회/저장 |
| **외부 API 호출** | API 도구로 서비스 연동 |
| **파일 조작** | 파일 시스템 도구로 읽기/쓰기 |

**💡 핵심**: LLM의 지식 한계를 극복하고 실제 작업을 수행할 수 있게 함

---

### 1. **랭체인 내장 도구**

- 랭체인은 다양한 기능을 수행할 수 있는 작업 도구를 지원

- Tavily 웹 검색 도구 (예시)

`(1) 도구(tool) 정의하기`

In [ ]:
from langchain_community.tools import TavilySearchResults

# 검색할 쿼리 설정
query = "스테이크와 어울리는 와인을 추천해주세요."

# Tavily 검색 도구 초기화 (최대 2개의 결과 반환)
web_search = TavilySearchResults(max_results=2)

# 웹 검색 실행
search_results = web_search.invoke(query)

# 검색 결과 출력
for result in search_results:
    print(result)  
    print("-" * 100)  

In [ ]:
# 도구 속성
print("자료형: ")
print(type(web_search))
print("-"*100)

print("name: ")
print(web_search.name)
print("-"*100)

print("description: ")
pprint(web_search.description)
print("-"*100)

print("schema: ")
pprint(web_search.args_schema.schema())
print("-"*100)

`(2) 도구(tool) 호출하기`

In [ ]:
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델 초기화
llm = ChatOpenAI(model="gpt-4.1-mini")

# 웹 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=[web_search])

In [ ]:
# 도구 호출이 필요 없는 LLM 호출을 수행
query = "안녕하세요."
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

In [ ]:
# 도구 호출이 필요한 LLM 호출을 수행
query = "한국시장에서 거래되는 ETF 종목의 수는 몇 개인가요?"
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

In [ ]:
tool_call = ai_msg.tool_calls[0]
tool_call

`(3) 도구(tool) 실행하기`

In [ ]:
### 방법 1: 직접 도구 호출 처리

# 이 방법은 AI 메시지에서 첫 번째 도구 호출을 가져와 직접 처리한다.
# 'args'를 사용하여 도구를 호출하고 결과를 얻는다.

tool_output = web_search.invoke(tool_call["args"])
print(f"{tool_call['name']} 호출 결과:")
print("-" * 100)
print(tool_output)

In [ ]:
### 방법 2: 도구 호출을 직접 사용하여 바로 ToolMessage 객체 생성

# 이 방법은 도구를 직접 호출하여 ToolMessage 객체를 생성한다.
# 가장 간단하고 직관적인 방법으로, LangChain의 추상화를 활용한다.

tool_message = web_search.invoke(tool_call)

print(type(tool_message))
print("-" * 100)
print(tool_message)

In [ ]:
pprint(tool_message.tool_call_id)

In [ ]:
pprint(tool_message.name)

In [ ]:
pprint(tool_message.content)

In [ ]:
ai_msg.tool_calls

In [ ]:
# batch 실행 - 도구 호출이 여러 개인 경우

# tool_messages = web_search.batch([tool_call])

tool_messages = web_search.batch(ai_msg.tool_calls)

print(tool_messages)
print("-" * 100)
pprint(tool_messages[0].content)

`(4) ToolMessage를 LLM에 전달하여 답변을 생성하기`

- **Agent**

    - **LLM(대규모 언어 모델)** 을 의사결정 엔진으로 사용하여 작업을 수행하는 시스템

    - 모델은 입력된 데이터를 분석하여 **맥락에 맞는 의사결정**을 수행

    - 시스템은 사용자의 요청을 이해하고 **적절한 해결책**을 제시

    - 복잡한 작업을 자동화하여 **업무 효율성**을 높일 수 있음 

- **create_agent** 

    - **create_agent**는 LangChain v1.0의 표준 에이전트 생성 함수

    - LangGraph를 기반으로 구축되어 **영속성, 스트리밍, Human-in-the-loop** 등의 기능을 자동 지원

    - **미들웨어**를 통한 유연한 커스터마이징 가능

In [ ]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

# 모델 초기화
llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

# 도구 목록
tools = [web_search]

# 에이전트 생성
agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt="당신은 사용자의 요청을 처리하는 AI Assistant입니다."
)

In [ ]:
# 에이전트 실행
response = agent.invoke(
    {"messages": [{"role": "user", "content": "한국시장에서 거래되는 ETF 종목의 수는 몇 개인가요?"}]},
)

# 에이전트 실행 결과 출력
pprint(response)

In [ ]:
for msg in response['messages']:
    msg.pretty_print()

---

### 2. **사용자 정의 도구** 

- **`@tool` 데코레이터**를 를 통해 사용자 정의 도구를 정의할 수 있음

- **함수와 스키마** 간 자동 연결로 도구 생성

- (예시) **네이버 개발자 API**를 사용하여 국내 뉴스 검색 도구를 정의


    - 네이버 개발자 API(https://developers.naver.com/)에서 인증 권한 취득 (회원 가입 및 애플리케이션 등록 필요)
    - 환경변수(.env)를 등록합니다. (**NAVER_CLIENT_ID**, **NAVER_CLIENT_SECRET**)

In [ ]:
# 환경 변수 로드
from dotenv import load_dotenv
load_dotenv()

`(1) 도구(tool) 정의하기`

In [ ]:
import requests, os
from langchain_core.tools import tool
from typing import Dict

@tool
def naver_news_search(
    query: str,
    ) -> Dict[Dict, int]:
    """네이버 검색 API를 사용하여 뉴스 검색 결과를 조회합니다.

    Args:
        query (str): 검색어

    Returns:
        Dict[Dict, int]: 검색 결과와 상태 코드  
    """


    url = "https://openapi.naver.com/v1/search/news.json"
    headers = {
        "X-Naver-Client-Id": os.getenv("NAVER_CLIENT_ID"),
        "X-Naver-Client-Secret": os.getenv("NAVER_CLIENT_SECRET")
    }
    params = {
        "query": query,
        "display": 100,
        }

    response = requests.get(url, headers=headers, params=params)

    return {
        "data": response.json(),
        "status_code": int(response.status_code)
    }  #type: ignore

In [ ]:
# 도구 속성
print("자료형: ")
print(type(naver_news_search))
print("-"*100)

print("name: ")
print(naver_news_search.name)
print("-"*100)

print("description: ")
pprint(naver_news_search.description)
print("-"*100)

print("schema: ")
pprint(naver_news_search.args_schema.schema())
print("-"*100)

In [ ]:
query = "딥시크 한국 진출"
search_result = naver_news_search.invoke(query)

print(search_result)

In [ ]:
search_result['data']['display'] # 검색 결과 개수

In [ ]:
search_result['data']['items'][:5]  # 검색 결과 목록

In [ ]:
# 검색 결과를 JSONL 형식으로 저장
with open("news_search_results.jsonl", "w", encoding="utf-8") as f:
    for item in search_result['data']['items']:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

In [ ]:
# 검색 결과를 가져오기
with open("news_search_results.jsonl", "r", encoding="utf-8") as f:
    
    news_items = []

    # JSONL 파일에서 각 줄을 읽어와서 JSON 객체로 변환
    for line in f:
        item = json.loads(line)
        news_items.append(item)


# 검색 결과 출력
for item in news_items[:3]:
    print(f"제목: {item['title']}")
    print(f"링크: {item['link']}")
    print(f"요약: {item['description']}")
    print(f"날짜: {item['pubDate']}")
    print("-" * 100)

`(2) LLM 도구 호출`

In [ ]:
# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[naver_news_search])

# 도구 호출이 필요한 LLM 호출을 수행
query = "딥시크 한국 진출 현황에 대해서 알려줘."
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

### **[실습]**

- 도구 호출 결과를 실행합니다. 
- 뉴스 검색 결과를 기반으로 답변하는 Agent를 정의하고 테스트합니다.

In [ ]:
# 여기에 코드를 작성하세요. 

---

### 3 **검색 결과 필터링** 

- 뉴스 데이터를 최근 기간으로 필터링

In [ ]:
import datetime
import re
from typing import List, Dict, Any, Optional
from email.utils import parsedate_to_datetime
from dateutil import parser as date_parser
import pytz

def filter_news_by_date_naver(
    news_data: List[Dict[str, Any]], 
    days_limit: int = 7
    ) -> List[Dict[str, Any]]:
    """
    네이버 뉴스 검색 API 결과를 최근 기간으로 필터링하는 함수
    
    Args:
        news_data (List[Dict]): 검색된 뉴스 데이터 리스트
        days_limit (int): 최근 몇 일 동안의 뉴스만 필터링할지 (기본값: 7일)
        
    Returns:
        List[Dict]: 최근 기간으로 필터링된 뉴스 데이터 리스트
    """
    if not news_data:
        return []
    
    filtered_news = []

    # timezone 정보가 없는(naive) 현재 날짜 사용
    current_date = datetime.datetime.now()
    cutoff_date = current_date - datetime.timedelta(days=days_limit)
    
    for i, news in enumerate(news_data):
        # 날짜 필드 찾기 (네이버 API는 'pubDate' 필드 사용)
        pub_date = news.get('pubDate')
        
        # 날짜 필드가 없는 경우 포함하지 않음
        if not pub_date:
            print(f"뉴스 {i+1}: 날짜 필드 없음")
            continue
            
        try:
            # 다양한 날짜 형식 처리
            news_date = parse_date_flexible(pub_date)
            
            if not news_date:
                print(f"뉴스 {i+1}: 날짜 형식 인식 불가 - {pub_date}")
                continue
            
            # timezone 정보가 있는(aware) 날짜를 naive로 변환
            if news_date.tzinfo is not None:
                news_date = news_date.replace(tzinfo=None)
                        
            # 설정한 기간 이내의 뉴스만 포함
            if news_date >= cutoff_date:
                filtered_news.append(news)
                
        except Exception as e:
            print(f"뉴스 {i+1}: 날짜 파싱 오류 - {str(e)}")
            continue
    
    return filtered_news

def parse_date_flexible(date_str: str) -> Optional[datetime.datetime]:
    """
    다양한 형식의 날짜 문자열을 파싱하는 함수
    
    Args:
        date_str (str): 파싱할 날짜 문자열
        
    Returns:
        Optional[datetime.datetime]: 파싱된 날짜, 파싱 실패 시 None
        
    Note:
        반환된 날짜는 timezone 정보가 포함될 수 있습니다 (offset-aware).
        비교하기 전에 timezone 정보를 제거해야 합니다.
    """
    # 빈 문자열 체크
    if not date_str or not isinstance(date_str, str):
        return None
        
    # RFC 822 형식 시도 (네이버 API 공식 문서 형식)
    try:
        return parsedate_to_datetime(date_str)
    except (ValueError, TypeError):
        pass
    
    # 일반적인 ISO 형식 시도
    try:
        return datetime.datetime.fromisoformat(date_str.replace('Z', '+00:00'))
    except (ValueError, TypeError):
        pass
    
    # 네이버 뉴스에서 자주 사용하는 형식 처리
    # 예: '2023년 10월 15일 오후 2시 30분'
    korean_pattern = r'(\d{4})년\s*(\d{1,2})월\s*(\d{1,2})일\s*(오전|오후)?\s*(\d{1,2})시\s*(\d{1,2})분'
    match = re.search(korean_pattern, date_str)
    if match:
        year, month, day, ampm, hour, minute = match.groups()
        hour = int(hour)
        if ampm == '오후' and hour < 12:
            hour += 12
        elif ampm == '오전' and hour == 12:
            hour = 0
        
        try:
            return datetime.datetime(int(year), int(month), int(day), hour, int(minute))
        except (ValueError, TypeError):
            pass
    
    # 다른 형식의 날짜 (YYYY-MM-DD, YYYY/MM/DD 등)
    # dateutil 라이브러리 사용 (pip install python-dateutil 필요)
    try:
        return date_parser.parse(date_str)
    except (ValueError, TypeError):
        pass
    
    # 원본 문자열에서 일자만 추출 시도 (마지막 방법)
    # 예: "Mon, 18 Mar 2025 14:30:45 +0900" 같은 형식에서 날짜만 추출
    date_patterns = [
        r'(\d{1,2})\s+(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s+(\d{4})',  # 18 Mar 2025
        r'(\d{4})-(\d{1,2})-(\d{1,2})',  # 2025-03-18
        r'(\d{1,2})/(\d{1,2})/(\d{4})'   # 3/18/2025 or 18/3/2025
    ]
    
    for pattern in date_patterns:
        match = re.search(pattern, date_str)
        if match:
            try:
                # 재구성된 간단한 형식으로 다시 시도
                simplified = ' '.join(match.groups())
                return date_parser.parse(simplified)
            except (ValueError, TypeError):
                pass
    
    # 로그에 파싱 실패한 날짜 형식 출력 (디버깅용)
    print(f"파싱 실패한 날짜 형식: {date_str}")
    
    # 모든 시도 실패
    return None


# 최근 10일 내 뉴스만 필터링
filtered_news = filter_news_by_date_naver(news_items, days_limit=10)
print("\n===== 필터링 결과 =====")
print(f"필터링된 뉴스 수: {len(filtered_news)}")
if result:
    for idx, news in enumerate(filtered_news, 1):
        print(f"\n[뉴스 {idx}]")
        print(f"제목: {news['title']}")
        print(f"날짜: {news['pubDate']}")
        print("-" * 30)
else:
    print("필터링된 뉴스가 없습니다.")

---

### 4 **뉴스 본문 수집** 

- 뉴스 링크를 사용하여, 본문 텍스트를 수집

In [ ]:
# 필터링된 뉴스 중 네이버 링크 가져오기
url = filtered_news[1]['link']

url

In [ ]:
# Data Loader - 웹페이지 데이터 가져오기
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader = WebBaseLoader(
    url,
    header_template={
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    },
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("media_and_head", "newsct_article _article_body")
        )
    ),
)

# 요청 속도 조절 (초당 요청 수)
loader.requests_per_second = 1 # 초에 1회 요청
docs = loader.load()

docs[0]

In [ ]:
pprint(docs[0].metadata)

In [ ]:
pprint(docs[0].page_content)

In [ ]:
# 모든 뉴스를 순회하면서 뉴스 본문을 가져오기
all_docs = []

for news in filtered_news:
    url = news['link']    
    docs = loader.load()
    all_docs.extend(docs)

print(f"총 {len(all_docs)}개의 뉴스 본문을 가져왔습니다.")
print("-" * 100)
print(all_docs[0].metadata)
print("-" * 100)
print(all_docs[0].page_content)
print("-" * 100)

---

### 5. **뉴스 본문 데이터 정제** 

`(1) 텍스트 정제`

In [ ]:
# 불필요한 공백 문자 제거
def clean_text(text: str) -> str:
    """
    텍스트에서 불필요한 공백, 줄바꿈 문자를 제거하는 함수
    """

    cleaned_text = str(text)

    # 연속된 줄바꿈을 하나로 통합
    cleaned_text = re.sub(r'\n+', '\n', cleaned_text)

    # 연속된 공백을 하나로 통합
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)

    # 줄 시작과 끝의 공백 제거
    cleaned_text = re.sub(r'^\s+|\s+$', '', cleaned_text, flags=re.MULTILINE)

    return cleaned_text
    

# 첫 번째 뉴스 본문에서 불필요한 공백 문자 제거
doc = all_docs[0]
doc.page_content = clean_text(doc.page_content)
print(doc.page_content)

`(2) 언론사, 기자 이름, 뉴스 본문 구분`

In [ ]:
from typing import Optional
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# 추출할 정보를 정의하는 Pydantic 모델 생성
class NewsArticle(BaseModel):
    """뉴스 기사에서 추출할 정보."""
    
    media_outlet: Optional[str] = Field(
        default=None, 
        description="뉴스 기사를 발행한 언론사 이름"
    )
    reporter: Optional[str] = Field(
        default=None, 
        description="기사를 작성한 기자의 이름"
    )
    title: Optional[str] = Field(
        default=None, 
        description="뉴스 기사의 제목"
    )
    content: Optional[str] = Field(
        default=None, 
        description="뉴스 기사의 본문 내용"
    )

# 추출 프롬프트 템플릿 정의
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 뉴스 기사에서 정보를 추출하는 전문가입니다. "
            "주어진 텍스트에서 관련 정보만 추출하세요. "
            "추출할 수 없는 정보가 있다면 해당 필드의 값으로 null을 반환하세요."
        ),
        ("human", "{text}")
    ]
)

# 모델 및 구조화된 출력 설정
llm = ChatOpenAI(model="gpt-4.1-mini") 
structured_llm = llm.with_structured_output(NewsArticle)

# 추출 파이프라인 생성
extraction_chain = prompt_template | structured_llm

# 사용 예시
result = extraction_chain.invoke({"text": doc.page_content})
print(result)

In [ ]:
result.model_dump()  # Pydantic 모델의 속성 및 값 출력

`(3) 모든 뉴스에 대해서 데이터 처리`

- 본문 공백 제거
- 추출된 데이터를 메타데이터에 추가

In [ ]:
processed_docs = []

# 모든 뉴스 본문에 대해 정보 추출 수행
for doc in all_docs:
    # 불필요한 공백 문자 제거
    doc.page_content = clean_text(doc.page_content)
    
    # 정보 추출 수행
    result = extraction_chain.invoke({"text": doc.page_content})
    
    # 추출된 정보를 메타데이터에 추가
    doc.metadata.update(result.model_dump())

    # 처리된 문서 리스트에 추가
    processed_docs.append(doc)

print(f"총 {len(processed_docs)}개의 뉴스 본문을 처리했습니다.")
print("-" * 100)
print(processed_docs[0].metadata)
print("-" * 100)
print(processed_docs[0].page_content)

In [ ]:
# 전처리 완료된 데이터를 pickle 파일로 저장
import pickle

with open("processed_news_articles.pkl", "wb") as f:
    pickle.dump(processed_docs, f)

In [ ]:
# pickle 파일에서 데이터 로드
with open("processed_news_articles.pkl", "rb") as f:
    loaded_docs = pickle.load(f)
    print(f"로드된 문서 수: {len(loaded_docs)}")

In [ ]:
loaded_docs[0].metadata

---

### **[실습 프로젝트]**

- 특정 주제에 대한 최신 뉴스 기사 수집하는 도구를 작성 (네이버 뉴스 검색 도구를 개선)
- 뉴스 검색 결과를 기반으로 답변하는 Agent 시스템을 구현합니다.

In [ ]:
# 여기에 코드를 작성하세요. 